In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../')
from seqmodel.bunch import Bunch

In [2]:
import time
import tensorflow as tf
import numpy as np
from seqmodel.model import rnn_module
from seqmodel.model import graph_util
from seqmodel.model.rnn_module import BasicRNNModule
from seqmodel.model.encoder import RNNEncoder
from seqmodel.model.decoder import RNNDecoder

In [14]:
tf.reset_default_graph()
batch_size = 4
max_time = 5
dim = 3
input_data = np.random.randn(max_time, batch_size, dim).astype(np.float32)
seq_len_data = np.array([4, 3, 2, 1], dtype=np.int32)
inputs = tf.constant(input_data, dtype=tf.float32)
seq_len = tf.constant(seq_len_data)
rnn_template = tf.make_template('share_rnn', tf.nn.dynamic_rnn, create_scope_now_=True)
# create share_rnn
share_rnn_opt = rnn_module.BasicRNNModule.default_opt()
share_rnn_opt.rnn_cell.cell_opt.num_units = 2
share_rnn_opt.logit=None
# create encoder
enc_rnn_opt = share_rnn_opt.clone()
enc = RNNEncoder(Bunch())(
    inputs, seq_len, BasicRNNModule(enc_rnn_opt), rnn_fn=rnn_template)
# create decoder
dec_rnn_opt = share_rnn_opt.clone()
dec_rnn_opt.logit = graph_util.default_logit_opt()
dec_opt = Bunch(RNNDecoder.default_opt(), init_with_encoder_state=True)
dec = RNNDecoder(dec_opt)(
    inputs, enc, seq_len, BasicRNNModule(dec_rnn_opt), rnn_fn=rnn_template)
for v in tf.trainable_variables():
    print(v.name)

share_rnn/rnn/basic_lstm_cell/weights:0
share_rnn/rnn/basic_lstm_cell/biases:0
basic_rnn_1/logit_w:0
basic_rnn_1/logit_b:0


In [15]:
sess_config = tf.ConfigProto(log_device_placement=False,
                             device_count={'GPU': 0})
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    fetch = Bunch(enc=Bunch(state=enc.final_state),
                  dec=Bunch(state=dec.final_state))
    result = sess.run(fetch)

In [16]:
result.enc

Bunch(state=LSTMStateTuple(c=array([[-0.53447843, -0.53694725],
       [ 0.29738659,  0.71461028],
       [ 0.10180426, -0.39652902],
       [ 0.06853295, -0.25919485]], dtype=float32), h=array([[-0.2342958 , -0.22444737],
       [ 0.18900128,  0.424936  ],
       [ 0.03910675, -0.09876543],
       [ 0.03483054, -0.10076944]], dtype=float32)))

In [17]:
result.dec

Bunch(state=LSTMStateTuple(c=array([[-0.58815289, -0.57701033],
       [ 0.31291491,  1.02366304],
       [ 0.19453263, -0.61040354],
       [ 0.14930536, -0.4588818 ]], dtype=float32), h=array([[-0.25378364, -0.23855136],
       [ 0.19234848,  0.54710186],
       [ 0.07544232, -0.13837117],
       [ 0.07717793, -0.16435823]], dtype=float32)))